# Custom CNN Classifier Evaluation

This notebook evaluates the custom CNN classifier on a held-out 20% validation set, and plots accuracy, precision, and loss curves.

**Environment:**
- Activate the `pygpu` Conda environment before running.

**Usage:**
```powershell
conda activate pygpu
jupyter notebook tests/test_custom_cnn_classifier_eval.ipynb
```

In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import precision_score, accuracy_score
import time
from tensorflow.keras.applications.efficientnet import preprocess_input

# Set paths
MODEL_PATH = os.path.join('..', 'src', 'image_processing', 'classifier1_models', 'custom_cnn_classifier_model.h5')
DATA_DIR = os.path.join('..', 'data', 'classifier1', 'augmented')
IMG_SIZE = (224, 224)
BATCH_SIZE = 8

# Load model
model = tf.keras.models.load_model(MODEL_PATH)

# Data generator for validation (20%)
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
val_gen = datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

# Evaluate
results = model.evaluate(val_gen, verbose=1, return_dict=True)
print('Validation Results:', results)

# Predict for precision/accuracy
y_true = val_gen.classes
y_pred_probs = model.predict(val_gen)
y_pred = np.argmax(y_pred_probs, axis=1)

acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, average='weighted')
print(f'Accuracy: {acc:.4f}, Precision: {prec:.4f}')

# EfficientNet model path
EFFNET_MODEL_PATH = os.path.join('..', 'src', 'image_processing', 'classifier1_models', 'efficientnet_classifier_model.h5')

# --- Evaluate Custom CNN ---
print('Evaluating Custom CNN...')
start_cnn = time.time()
model_cnn = tf.keras.models.load_model(MODEL_PATH)
cnn_results = model_cnn.evaluate(val_gen, verbose=1, return_dict=True)
cnn_pred_probs = model_cnn.predict(val_gen)
cnn_pred = np.argmax(cnn_pred_probs, axis=1)
cnn_acc = accuracy_score(y_true, cnn_pred)
cnn_prec = precision_score(y_true, cnn_pred, average='weighted')
end_cnn = time.time()
cnn_time = end_cnn - start_cnn
cnn_size = os.path.getsize(MODEL_PATH) / 1024 / 1024  # MB
print(f'Custom CNN - Accuracy: {cnn_acc:.4f}, Precision: {cnn_prec:.4f}, Time: {cnn_time:.2f}s, Size: {cnn_size:.2f}MB')

# --- Evaluate EfficientNet ---
print('Evaluating EfficientNet-B0...')
start_eff = time.time()
model_eff = tf.keras.models.load_model(EFFNET_MODEL_PATH)
# Use same validation set, but with EfficientNet preprocessing
datagen_eff = ImageDataGenerator(preprocessing_function=preprocess_input, validation_split=0.2)
val_gen_eff = datagen_eff.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)
eff_results = model_eff.evaluate(val_gen_eff, verbose=1, return_dict=True)
eff_pred_probs = model_eff.predict(val_gen_eff)
eff_pred = np.argmax(eff_pred_probs, axis=1)
eff_acc = accuracy_score(y_true, eff_pred)
eff_prec = precision_score(y_true, eff_pred, average='weighted')
end_eff = time.time()
eff_time = end_eff - start_eff
eff_size = os.path.getsize(EFFNET_MODEL_PATH) / 1024 / 1024  # MB
print(f'EfficientNet-B0 - Accuracy: {eff_acc:.4f}, Precision: {eff_prec:.4f}, Time: {eff_time:.2f}s, Size: {eff_size:.2f}MB')

# --- Comparison ---
print('\n--- Model Comparison ---')
print(f"Custom CNN:     Acc={cnn_acc:.4f}, Prec={cnn_prec:.4f}, Time={cnn_time:.2f}s, Size={cnn_size:.2f}MB")
print(f"EfficientNet-B0: Acc={eff_acc:.4f}, Prec={eff_prec:.4f}, Time={eff_time:.2f}s, Size={eff_size:.2f}MB")

# Percentage differences
acc_diff = 100 * (eff_acc - cnn_acc) / cnn_acc if cnn_acc else 0
prec_diff = 100 * (eff_prec - cnn_prec) / cnn_prec if cnn_prec else 0
time_diff = 100 * (eff_time - cnn_time) / cnn_time if cnn_time else 0
size_diff = 100 * (eff_size - cnn_size) / cnn_size if cnn_size else 0
print(f"\nAccuracy diff: {acc_diff:+.2f}%\nPrecision diff: {prec_diff:+.2f}%\nTime diff: {time_diff:+.2f}%\nSize diff: {size_diff:+.2f}%")

# --- Plot loss and accuracy curves from training history (if available) ---
import json
plt.figure(figsize=(12,4))
for i, (name, path) in enumerate([
    ("Custom CNN", os.path.join('..', 'src', 'image_processing', 'classifier1_models', 'custom_cnn_classifier_history.json')),
    ("EfficientNet-B0", os.path.join('..', 'src', 'image_processing', 'classifier1_models', 'efficientnet_classifier_history.json'))
]):
    if os.path.exists(path):
        with open(path, 'r') as f:
            history = json.load(f)
        plt.subplot(2,2,1+i*2)
        plt.plot(history['loss'], label='Train Loss')
        plt.plot(history['val_loss'], label='Val Loss')
        plt.legend(); plt.title(f'{name} Loss Curve')
        plt.subplot(2,2,2+i*2)
        plt.plot(history['accuracy'], label='Train Acc')
        plt.plot(history['val_accuracy'], label='Val Acc')
        plt.legend(); plt.title(f'{name} Accuracy Curve')
plt.tight_layout()
plt.show()

Found 1027 images belonging to 2 classes.
129/129 [==============================] - 2s 14ms/step - loss: 0.1538 - accuracy: 0.9727
Validation Results: {'loss': 0.15382526814937592, 'accuracy': 0.972736120223999}
129/129 [==============================] - 2s 12ms/step
Accuracy: 0.9727, Precision: 0.9728
Evaluating Custom CNN...
129/129 [==============================] - 2s 12ms/step
Custom CNN - Accuracy: 0.9727, Precision: 0.9728, Time: 4.05s, Size: 78.06MB
Evaluating EfficientNet-B0...


ValueError: No model config found in the file at <tensorflow.python.platform.gfile.GFile object at 0x000001A0DC628A30>.